## FunkSVD
У алгортима SVD есть два серьёзных минуса:
1. Он медленный, а значит чтобы применить к реальной матрице пользователь-элемент в которой будут тычсичи (если не миллионы) строк и колонок, будет почти невозможно.
2. Проблема разряженности, в реальности у каждого пользователя будет максимум 1% оцененных товаров, и заполнять остальные товары среднеми значениями (используя даже продвинутую технику базисных предикатов) не очень правильно.

Чтобы решить эту проблем был разработан FunkSVD. FunkSVD - принимает на вход не матрицу пользователей-товаров.А список всех оценок, то есть он принимает только выставленные оценки, и ему не нужно заполнять пропуски. Во вторых мы работаем со списком, а не с бесконечной матрицей, это должно повысить производительность.

In [30]:
import requests
import time
import pandas as pd
import surprise as surprise
import numpy as np
from decimal import Decimal
from scipy.sparse import coo_matrix
from scipy import sparse
from tqdm import tqdm
import random
from decimal import Decimal
from collections import defaultdict
import math
import sys

In [31]:
users_ratings = pd.read_csv("ratings.csv")
print(users_ratings.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [32]:
users_ratings = users_ratings.drop('timestamp', axis=1)
# movie_data = pd.merge(ratings_data, movie_names, on='movieId')
users_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [33]:
def my_calculate_all_movies_mean(ratings):
    avg = ratings['rating'].sum() / ratings.shape[0]
    return Decimal(avg)

In [34]:
def my_initialize_factors(ratings, k=25):
        user_ids = set(ratings['userId'].values)
        movie_ids = set(ratings['movieId'].values)
        item_counts = ratings[['movieId', 'rating']].groupby('movieId').count()
        item_counts = item_counts.reset_index()
        print("ITEM COUNTS")
        print(item_counts)

        item_sum = ratings[['movieId', 'rating']].groupby('movieId').sum()
        item_sum = item_sum.reset_index()
        print(item_sum)

        u_inx = {r: i for i, r in enumerate(user_ids)}
        i_inx = {r: i for i, r in enumerate(movie_ids)}
#         i_inx строк, k колонок
        item_factors = np.full((len(i_inx), k), Decimal(0.1))
        user_factors = np.full((len(u_inx), k), Decimal(0.1))
        all_movies_mean = my_calculate_all_movies_mean(ratings)
        print("ALL MOVIES MEAN")
        print(all_movies_mean)
        user_bias = defaultdict(lambda: 0)
        item_bias = defaultdict(lambda: 0)
        
        return u_inx, i_inx, all_movies_mean, user_bias, item_bias, item_factors, user_factors

In [35]:
# стохастический градиентный спуск добавляет только один фактор (один для пользователя и один для элемента) за раз, новые факторы вычисляются на основе предыдущих
def my_stocastic_gradient_descent(factor, index_randomized, ratings, u_inx,i_inx,user_factors,item_factors,all_movies_mean,user_bias,item_bias):

#     Параметры нужно подбирать чтобы RMSE была минимальной
    #         скорость обучения
        lr = Decimal(0.2)
#     скорость обучения для отклонения
        b_lr = Decimal(0.5)
# регуляризация - сила штрафа
        r = Decimal(0.3)
    # регуляризация для отклонения - сила штрафа для отклонения
        bias_r = Decimal(0.3)

        for inx in index_randomized:
#             получаем строку - [user_index, movie_index, rating]
            rating_row = ratings[inx]

#     получаем индекс пользователя

            u = u_inx[rating_row[0]]
    #  получаем индекс элемента
            i = i_inx[rating_row[1]]
#         получаем рейтинг
            rating = Decimal(rating_row[2])

#     получаем ошибку
            err = (rating - my_predict(u, i,user_factors,item_factors,all_movies_mean,user_bias,item_bias))

    #     корректируем отклонения
            user_bias[u] += b_lr * (err - bias_r * user_bias[u])
            item_bias[i] += b_lr * (err - bias_r * item_bias[i])

#             получаем текущий фактор пользователя
            user_fac = user_factors[u][factor]
#             получаем текущий фактор элемента
            item_fac = item_factors[i][factor]

#     изменяем текущий фактор пользователя
            user_factors[u][factor] += lr * (err * item_fac- r * user_fac)
#     изменяем текущий фактор элемента                                                  
            item_factors[i][factor] += lr * (err * user_fac- r * item_fac)
                                                  
        return my_calculate_rmse(ratings, factor,u_inx,i_inx,user_factors,item_factors,all_movies_mean,user_bias,item_bias), user_bias,item_bias,user_factors,item_factors

In [36]:
#need to test
def my_calculate_rmse(ratings, factor,u_inx,i_inx,user_factors,item_factors,all_movies_mean,user_bias,item_bias):

        # row - [user_index, movie_index, rating]  
        def difference(row):
            
            user_index = u_inx[row[0]]
            item_index = i_inx[row[1]]

#             Вычисляем значение факторов для пользователя и фильма
            pq = np.dot(item_factors[item_index][:factor + 1], user_factors[user_index][:factor + 1].T)
#     Вычисляем смещение
            b_ui = all_movies_mean + user_bias[user_index] + item_bias[item_index]
#     получаем прогноз
            prediction = b_ui + pq
#     Вычитаем из прогноза реальную оценку и возводим в квадрат
            MSE = (prediction - Decimal(row[2])) ** 2
            return MSE

#         print(np.apply_along_axis(difference, 1, ratings))
#         np.apply_along_axis(difference, 1, ratings) список ошибок для каждой ошибки
# проссумируем их, разделим на количество строк и получим среднюю ошибку
# Извлечём квадрат из средней ошибки, так как раньше мы возвели в квадрат.
        squared = np.apply_along_axis(difference, 1, ratings).sum()
        print("RMSE: ",math.sqrt(squared / ratings.shape[0]))
        return math.sqrt(squared / ratings.shape[0])

In [37]:
def my_predict(user_index, item_index,user_factors,item_factors,all_movies_mean,user_bias,item_bias):
        
#         item_factors[item_index] список факторов фильма, user_factors[user_index].T список факторов пользователя (транспонированный)
# перемножаем строку на столбец и получаем одно число (pq)
        pq = np.dot(item_factors[item_index], user_factors[user_index].T)
#     расчитываем отклонение
        b_ui = all_movies_mean + user_bias[user_index] + item_bias[item_index]
        prediction = b_ui + pq

#         нормализация прогноза от 1 до 10 включительно
        if prediction > 10:
            prediction = 10
        elif prediction < 1:
            prediction = 1
        return prediction

In [38]:
#iterations:номер итерации,
#градиентный спуск завершается когда достигнут придел итераций, или приращение становится очень маленьким < 0.01
def my_finished(iterations, last_err, current_err,MAX_ITERATIONS=100):
        if (iterations >= MAX_ITERATIONS):
            print("FULL")
        
        if iterations >= MAX_ITERATIONS or last_err - current_err < 0.001:
            return True
        else:
            return False

In [41]:
def my_train(ratings_df, k=40):
        
        u_inx, i_inx, all_movies_mean, user_bias, item_bias, item_factors, user_factors = my_initialize_factors(ratings_df, k)

        ratings = ratings_df[['userId', 'movieId', 'rating']].values
        index_randomized = random.sample(range(0, len(ratings)), (len(ratings) - 1))

        for factor in tqdm(range(k)):
            iterations = 0
            last_err = sys.maxsize
            iteration_err = sys.maxsize
            finished = False
            
            while not finished:
                iteration_err,user_bias,item_bias,user_factors,item_factors = my_stocastic_gradient_descent(factor,index_randomized, ratings, u_inx,i_inx,user_factors,item_factors,all_movies_mean,user_bias,item_bias)

                iterations += 1
                
                finished = my_finished(iterations,last_err,iteration_err)
                last_err = iteration_err

                

        return user_factors,item_factors,all_movies_mean,user_bias,item_bias,u_inx,i_inx

In [42]:
user_factors,item_factors,all_movies_mean,user_bias,item_bias,u_inx,i_inx = my_train(users_ratings,k=100)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

ITEM COUNTS
      movieId  rating
0           1     215
1           2     110
2           3      52
3           4       7
4           5      49
...       ...     ...
9719   193581       1
9720   193583       1
9721   193585       1
9722   193587       1
9723   193609       1

[9724 rows x 2 columns]
      movieId  rating
0           1   843.0
1           2   377.5
2           3   169.5
3           4    16.5
4           5   150.5
...       ...     ...
9719   193581     4.0
9720   193583     3.5
9721   193585     3.5
9722   193587     3.5
9723   193609     4.0

[9724 rows x 2 columns]
ALL MOVIES MEAN
3.50155698361696199327752765384502708911895751953125
RMSE:  1.3927425882446731
RMSE:  1.3863378040730467
RMSE:  1.3847773657579228


  1%|▊                                                                                 | 1/100 [00:15<24:49, 15.04s/it]

RMSE:  1.3842220354414567
RMSE:  1.3777213954414584


  2%|█▋                                                                                | 2/100 [00:22<17:42, 10.84s/it]

...

RMSE:  1.0620497391623895
RMSE:  1.0618184348228867


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [18:55<00:15, 15.43s/it]

RMSE:  1.0618131080968929
RMSE:  1.061653267656159


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [19:11<00:00, 11.52s/it]

RMSE:  1.0616481203712327


In [43]:
user_index = u_inx[1]
item_index = i_inx[47]
print(my_predict(user_index, item_index,user_factors,item_factors,all_movies_mean,user_bias,item_bias))

4.651974008705397155115448911


In [44]:
user_movie_rating = users_ratings.pivot_table(index='userId', columns='movieId', values='rating')
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
for rowIndex, row in tqdm(user_movie_rating.iterrows()): #iterate over rows
    for columnIndex, value in row.items():
        user_movie_rating.loc[rowIndex][columnIndex]=my_predict(user_index, item_index,user_factors,item_factors,all_movies_mean,user_bias,item_bias)

610it [07:40,  1.33it/s]


In [46]:
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
2,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
3,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
4,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
5,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974


In [47]:
user_movie_rating.to_csv('FunkSVD_user_movie_rating.csv',index=False)

In [48]:
loaded_data = pd.read_csv("FunkSVD_user_movie_rating.csv")
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
2,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
3,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
4,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974
5,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,...,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974,4.651974


In [52]:
user_movie_rating.loc[1,47]

4.651974008705397